## 実践Python スクレイピングと可視化

今日お伝えしたことをスクレイピングと可視化をしながら学びます

### すること
1. `download` ディレクトリに格納している開店閉店情報をスクレイピング
1. データを前処理
1. 可視化

### しないこと
- html コンテンツのダウンロード
    - 時間がかかるし、みんながアクセスすると迷惑になるのでダウンロードしておきました。download ディレクトリに格納しています
    - ダウンロードのスクリプトは `src/download.py` に書いています


## ダウンロードした 開店閉店情報htmlをスクレイピング

- リスト化されている開店閉店情報から、日付け、カテゴリー、店名、URLを取得する
- python で scraping する時のライブラリはだいたい `BeautifulSoup` を使う
    - [Beautiful Soup Documentation — Beautiful Soup 4.9.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)



In [54]:
# 使いたいライブラリやモジュールをインポート
from bs4 import BeautifulSoup

### contents を 読み込む
- 今日やったところ
    - [ ] 関数
    - [ ] メソッド

In [6]:
# htmlファイルを開いて、中身を読み込む
# open関数：引数に渡されたファイルパスのファイルを読み込んでファイル風オブジェクトを返す関数
# .read() メソッド: ファイル風オブジェクトがファイルの中身(データ)を文字列で返す
html = open("../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_1_.html").read()
print(type(html))


<class 'str'>


### コンテンツを スクレイピングするために beautifulsoup オブジェクトに変換
- 今日やったところ
    - [ ] クラス
    - [ ] クラスのインスタンス化
    - [ ] クラスの初期値設定

In [7]:
# BeautifulSoupクラスを初期化してBeautifulSoupインスタンスオブジェクト作る
soup = BeautifulSoup(
    html, # スクレイピングしたいコンテンツ
    'html.parser'  #パーサーの指定
)
type(soup)

bs4.BeautifulSoup

### beautifulsoup オブジェクトのメソッドを使って情報を抽出
今回は、CSSの情報を元にデータを抽出

- 今日やったところ
    - [ ] オブジェクト
    - [ ] オブジェクトが持つメソッドを確認 dir

In [8]:
# soup オブジェクトが持つアトリビュートを確認 
# 各アトリビュートの説明は https://www.crummy.com/software/BeautifulSoup/bs4/ をみてね
dir(soup)

['ASCII_SPACES',
 'DEFAULT_BUILDER_FEATURES',
 'NO_PARSER_SPECIFIED_WARNING',
 'ROOT_TAG_NAME',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_check_markup_is_url',
 '_decode_markup',
 '_feed',
 '_find_all',
 '_find_one',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_linkage_fixer',
 '_most_recent_element',
 '_namespaces',
 '_popToTag',
 '_should_pretty_print',
 'append',
 'attrs',
 'builder',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator'

1. `_category_kantou_koushinetsu_tokyo_page_1_.html`をブラウザで開いて、デベロッパーツールを開く
1. inspector ボタンを押してブラウザ上で必要な情報の詳細を得る。
1. お店1つずつの情報が `article` というタグにまとまってたのでこれを[css-selector](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors) をつかって取得する．
    - `.select()` メソッドを使う
    - 引数にはタグ名、クラス名、IDなどを渡す
    



In [10]:
# 全ての article タグを持つデータをリストで返す
soup.select("article")

[<article class="list col-md-12 post-287548 post type-post status-publish format-standard has-post-thumbnail hentry category-coffie category-tokyo category-close category-kantou_koushinetsu category-restaurant">
 <a class="post_links" href="https://kaiten-heiten.com/ginza-renoir-ginza2chome/" title="【閉店】喫茶室ルノアール 銀座2丁目店">
 <div class="list-block">
 <div class="post_thumb" style="background-image: url('https://i2.wp.com/kaiten-heiten.com/wp-content/uploads/images/2021/09/004-6.jpg?fit=640%2C399&amp;ssl=1')"><span> </span></div>
 <div class="list-text">
 <span class="post_time"><i class="icon icon-clock"></i> 2021-09-01</span>
 <span class="post_cat"><i class="icon icon-folder"></i> コーヒーショップ, 東京, 閉店情報, 関東・甲信越, 飲食店</span> <h3 class="list-title post_ttl">【閉店】喫茶室ルノアール 銀座2丁目店</h3>
 </div>
 </div>
 </a>
 </article>,
 <article class="list col-md-12 post-287529 post type-post status-publish format-standard has-post-thumbnail hentry category-shop category-tokyo category-clothing-shop category-clo

In [11]:
# 最初の1つだけほしい場合は、リストのインデックス指定か、`.select_one()` メソッドを使う
soup.select("article")[0]
soup.select_one("article") 

<article class="list col-md-12 post-287548 post type-post status-publish format-standard has-post-thumbnail hentry category-coffie category-tokyo category-close category-kantou_koushinetsu category-restaurant">
<a class="post_links" href="https://kaiten-heiten.com/ginza-renoir-ginza2chome/" title="【閉店】喫茶室ルノアール 銀座2丁目店">
<div class="list-block">
<div class="post_thumb" style="background-image: url('https://i2.wp.com/kaiten-heiten.com/wp-content/uploads/images/2021/09/004-6.jpg?fit=640%2C399&amp;ssl=1')"><span> </span></div>
<div class="list-text">
<span class="post_time"><i class="icon icon-clock"></i> 2021-09-01</span>
<span class="post_cat"><i class="icon icon-folder"></i> コーヒーショップ, 東京, 閉店情報, 関東・甲信越, 飲食店</span> <h3 class="list-title post_ttl">【閉店】喫茶室ルノアール 銀座2丁目店</h3>
</div>
</div>
</a>
</article>

In [17]:
# 各要素は bs4.element.Tag オブジェクト
type(soup.select("article")[0])


bs4.element.Tag

In [21]:
# オブジェクトなんだからメソッド持ってるよね？と確認すると使えそうなやつがたくさん用意されている
dir(soup.select("article")[0])


['__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_find_all',
 '_find_one',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_should_pretty_print',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'decomposed',
 'descendants',
 'encode',
 'encode_contents',
 'extend',
 'extract',
 'fetchNextSiblings',
 'fetchParents',
 'fetchPrevious',
 'fetchPreviousSiblings',
 '

article タグの中を確認してみる

```html
<article class="list col-md-12 post-287548 post type-post status-publish format-standard has-post-thumbnail hentry category-coffie category-tokyo category-close category-kantou_koushinetsu category-restaurant">
    <a class="post_links" href="https://kaiten-heiten.com/ginza-renoir-ginza2chome/" title="【閉店】喫茶室ルノアール 銀座2丁目店">
        <div class="list-block">
            <div class="post_thumb" style="background-image: url('https://i2.wp.com/kaiten-heiten.com/wp-content/uploads/images/2021/09/004-6.jpg?fit=640%2C399&amp;ssl=1')"><span> </span></div>
            <div class="list-text">
                <span class="post_time"><i class="icon icon-clock"></i> 2021-09-01</span>
                <span class="post_cat"><i class="icon icon-folder"></i> コーヒーショップ, 東京, 閉店情報, 関東・甲信越, 飲食店</span> 
                <h3 class="list-title post_ttl">【閉店】喫茶室ルノアール 銀座2丁目店</h3>
            </div>
        </div>
    </a>
</article>
```
- `a` タグの中で
    - `title` アトリビュートで店名
    - `<span class="post_time">` で日付け
    - `<span class="post_cat">` でカテゴリ
- `class=` 属性でカテゴリーリスト

In [36]:
a = soup.select("article")[0].select_one("a")
a.get("title")


'【閉店】喫茶室ルノアール 銀座2丁目店'

In [39]:
a.select_one("span.post_time").get_text()

' 2021-09-01'

In [40]:
a.select_one("span.post_cat").get_text()

' コーヒーショップ, 東京, 閉店情報, 関東・甲信越, 飲食店'

- 今日やったところ
    - [ ] 内包表記（ちょっとバージョンアップ版）


In [43]:
# class の中身全部がカテゴリーではない
soup.select("article")[0].get("class")

['list',
 'col-md-12',
 'post-287548',
 'post',
 'type-post',
 'status-publish',
 'format-standard',
 'has-post-thumbnail',
 'hentry',
 'category-coffie',
 'category-tokyo',
 'category-close',
 'category-kantou_koushinetsu',
 'category-restaurant']

In [45]:
# 文字列が持つ find メソッドを使うと、引数に渡した文字列が入っているならばその位置を、入っていなければ-1を返す
'category-coffie'.find("category")

0

In [46]:
'post-287548'.find("category")

-1

In [49]:
# これをcategoryだけのリストを作成
l = list()
for cat in soup.select("article")[0].get("class"):
    if cat.find("category") > -1:
        l.append(cat)
l 
        

['category-coffie',
 'category-tokyo',
 'category-close',
 'category-kantou_koushinetsu',
 'category-restaurant']

In [50]:
# 条件付きリスト内包表記
l = [cat for cat in soup.select("article")[0].get("class") if cat.find("category") > -1]
l 

['category-coffie',
 'category-tokyo',
 'category-close',
 'category-kantou_koushinetsu',
 'category-restaurant']

In [51]:
# いままでのをまとめて関数にする
# 返り値は辞書に入れる

def get_details(soup):
    """
    soup: soup.select_one("article") もしくは、soup.select("article")[0] を想定
    """
    soup_a = soup.select_one("a")
    name = soup_a.get("title")
    post_time = soup_a.select_one("span.post_time").get_text()
    url = soup_a.get("href")
    category = soup_a.select_one("span.post_cat").get_text()

    category_tag = [cat for cat in  soup.get("class") if cat.find("category") > -1]# 

    return {
        "name":name, 
        "post_time":post_time,
        "url":url,
        "category":category,
        "category_tag":category_tag}

In [52]:
# テスト
get_details(soup.select("article")[0])

{'name': '【閉店】喫茶室ルノアール 銀座2丁目店',
 'post_time': ' 2021-09-01',
 'url': 'https://kaiten-heiten.com/ginza-renoir-ginza2chome/',
 'category': ' コーヒーショップ, 東京, 閉店情報, 関東・甲信越, 飲食店',
 'category_tag': ['category-coffie',
  'category-tokyo',
  'category-close',
  'category-kantou_koushinetsu',
  'category-restaurant']}

In [53]:
# これを article タグのデータ全てに適用する
[get_details(article) for article in soup.select("article")]

[{'name': '【閉店】喫茶室ルノアール 銀座2丁目店',
  'post_time': ' 2021-09-01',
  'url': 'https://kaiten-heiten.com/ginza-renoir-ginza2chome/',
  'category': ' コーヒーショップ, 東京, 閉店情報, 関東・甲信越, 飲食店',
  'category_tag': ['category-coffie',
   'category-tokyo',
   'category-close',
   'category-kantou_koushinetsu',
   'category-restaurant']},
 {'name': '【閉店】Right-on（ライトオン）上野店',
  'post_time': ' 2021-09-01',
  'url': 'https://kaiten-heiten.com/right-on-ueno/',
  'category': ' 小売店, 東京, 洋服, 閉店情報, 関東・甲信越',
  'category_tag': ['category-shop',
   'category-tokyo',
   'category-clothing-shop',
   'category-close',
   'category-kantou_koushinetsu']},
 {'name': '【閉店】丸亀製麺 天王洲アイル店',
  'post_time': ' 2021-09-01',
  'url': 'https://kaiten-heiten.com/marugameseimen-tennouzu-isle/',
  'category': ' そば・うどん店, 東京, 開店情報, 関東・甲信越, 飲食店',
  'category_tag': ['category-soba_udon',
   'category-tokyo',
   'category-open',
   'category-kantou_koushinetsu',
   'category-restaurant']},
 {'name': '【閉店】フジヤ時計店',
  'post_time': ' 2021-08-30',


In [59]:
# html ファイルを引数にとってスクレイピングして辞書リストを返す関数を定義
def fetch_all(f):
    html = open(f).read()
    soup = BeautifulSoup(html, 'html.parser')
    return [get_details(article) for article in soup.select("article")]
    


In [60]:
# test
fetch_all("../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_1_.html")


[{'name': '【閉店】喫茶室ルノアール 銀座2丁目店',
  'post_time': ' 2021-09-01',
  'url': 'https://kaiten-heiten.com/ginza-renoir-ginza2chome/',
  'category': ' コーヒーショップ, 東京, 閉店情報, 関東・甲信越, 飲食店',
  'category_tag': ['category-coffie',
   'category-tokyo',
   'category-close',
   'category-kantou_koushinetsu',
   'category-restaurant']},
 {'name': '【閉店】Right-on（ライトオン）上野店',
  'post_time': ' 2021-09-01',
  'url': 'https://kaiten-heiten.com/right-on-ueno/',
  'category': ' 小売店, 東京, 洋服, 閉店情報, 関東・甲信越',
  'category_tag': ['category-shop',
   'category-tokyo',
   'category-clothing-shop',
   'category-close',
   'category-kantou_koushinetsu']},
 {'name': '【閉店】丸亀製麺 天王洲アイル店',
  'post_time': ' 2021-09-01',
  'url': 'https://kaiten-heiten.com/marugameseimen-tennouzu-isle/',
  'category': ' そば・うどん店, 東京, 開店情報, 関東・甲信越, 飲食店',
  'category_tag': ['category-soba_udon',
   'category-tokyo',
   'category-open',
   'category-kantou_koushinetsu',
   'category-restaurant']},
 {'name': '【閉店】フジヤ時計店',
  'post_time': ' 2021-08-30',


## `download` ディレクトリに入っている全ファイルをスクレイピング

### ファイルリストを取得

`download` ホルダにダウンロードしている全htmlファイルに get_details関数を適用してデータを取得
- 今日やったところ
    - [ ] 組み込みクラスのインポートとインスタンス化


In [56]:
# glob --- Unix 形式のパス名のパターン展開 https://docs.python.org/ja/3/library/glob.html

import glob 
# 【閉店】ディレクトリに入っているファイルをワイルドカードで指定。当てはまるフルパスをリストで返す
glob.glob("../download/【閉店】/*.html")


['../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_33_.html',
 '../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_54_.html',
 '../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_56_.html',
 '../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_44_.html',
 '../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_53_.html',
 '../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_14_.html',
 '../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_30_.html',
 '../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_47_.html',
 '../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_19_.html',
 '../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_35_.html',
 '../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_16_.html',
 '../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_48_.html',
 '../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_22_.html',
 '../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_39_.html',
 '../download/【閉店】/_

In [65]:
# このまますると、二重リストになる
[fetch_all(f) for f in glob.glob("../download/【閉店】/*.html")[:3]]

    


[[{'name': '【閉店】串カツ田中 大森店',
   'post_time': ' 2021-02-16',
   'url': 'https://kaiten-heiten.com/kushi-tanaka-omori/',
   'category': ' 串揚げ・串焼き, 居酒屋, 東京, 閉店情報, 関東・甲信越, 飲食店',
   'category_tag': ['category-kushiage',
    'category-izakaya',
    'category-tokyo',
    'category-close',
    'category-kantou_koushinetsu',
    'category-restaurant']},
  {'name': '【閉店】ドラッグストアスマイル下井草駅前店',
   'post_time': ' 2021-02-15',
   'url': 'https://kaiten-heiten.com/drug-smile-igusaekimae/',
   'category': ' ドラッグストア・調剤薬局, 小売店, 東京, 閉店情報, 関東・甲信越',
   'category_tag': ['category-drug-store',
    'category-shop',
    'category-tokyo',
    'category-close',
    'category-kantou_koushinetsu']},
  {'name': '【閉店】RATIO&C',
   'post_time': ' 2021-02-15',
   'url': 'https://kaiten-heiten.com/ratio-c/',
   'category': ' 東京, 自転車・販売・買取・修理, 車・不動産, 閉店情報, 関東・甲信越',
   'category_tag': ['category-tokyo',
    'category-bicycle-shop',
    'category-car-estate',
    'category-close',
    'category-kantou_koushinetsu']},
  {'name'

In [70]:
# こういう時は flatten 
# itertools に入っている chain.from_iterable クラスにネストしたデータを渡すとイテレータオブジェクトを返す
import itertools
itertools.chain.from_iterable([fetch_all(f) for f in glob.glob("../download/【閉店】/*.html")[:3]])


In [71]:
# イテレータオブジェクトは list 関数に渡すと、リストになる
list(itertools.chain.from_iterable([fetch_all(f) for f in glob.glob("../download/【閉店】/*.html")[:3]]))

[{'name': '【閉店】串カツ田中 大森店',
  'post_time': ' 2021-02-16',
  'url': 'https://kaiten-heiten.com/kushi-tanaka-omori/',
  'category': ' 串揚げ・串焼き, 居酒屋, 東京, 閉店情報, 関東・甲信越, 飲食店',
  'category_tag': ['category-kushiage',
   'category-izakaya',
   'category-tokyo',
   'category-close',
   'category-kantou_koushinetsu',
   'category-restaurant']},
 {'name': '【閉店】ドラッグストアスマイル下井草駅前店',
  'post_time': ' 2021-02-15',
  'url': 'https://kaiten-heiten.com/drug-smile-igusaekimae/',
  'category': ' ドラッグストア・調剤薬局, 小売店, 東京, 閉店情報, 関東・甲信越',
  'category_tag': ['category-drug-store',
   'category-shop',
   'category-tokyo',
   'category-close',
   'category-kantou_koushinetsu']},
 {'name': '【閉店】RATIO&C',
  'post_time': ' 2021-02-15',
  'url': 'https://kaiten-heiten.com/ratio-c/',
  'category': ' 東京, 自転車・販売・買取・修理, 車・不動産, 閉店情報, 関東・甲信越',
  'category_tag': ['category-tokyo',
   'category-bicycle-shop',
   'category-car-estate',
   'category-close',
   'category-kantou_koushinetsu']},
 {'name': '【閉店】FLIPPER’S（フリッパーズ） 表参道店

In [72]:
# ここまでを関数化
# html のファイルリストを渡したら辞書データのリストを返す関数を定義

def html_to_data(file_list):
    return list(itertools.chain.from_iterable([fetch_all(f) for f in file_list]))

In [73]:
# test 
heiten_files = glob.glob("../download/【閉店】/*.html")
html_to_data(heiten_files)

[{'name': '【閉店】串カツ田中 大森店',
  'post_time': ' 2021-02-16',
  'url': 'https://kaiten-heiten.com/kushi-tanaka-omori/',
  'category': ' 串揚げ・串焼き, 居酒屋, 東京, 閉店情報, 関東・甲信越, 飲食店',
  'category_tag': ['category-kushiage',
   'category-izakaya',
   'category-tokyo',
   'category-close',
   'category-kantou_koushinetsu',
   'category-restaurant']},
 {'name': '【閉店】ドラッグストアスマイル下井草駅前店',
  'post_time': ' 2021-02-15',
  'url': 'https://kaiten-heiten.com/drug-smile-igusaekimae/',
  'category': ' ドラッグストア・調剤薬局, 小売店, 東京, 閉店情報, 関東・甲信越',
  'category_tag': ['category-drug-store',
   'category-shop',
   'category-tokyo',
   'category-close',
   'category-kantou_koushinetsu']},
 {'name': '【閉店】RATIO&C',
  'post_time': ' 2021-02-15',
  'url': 'https://kaiten-heiten.com/ratio-c/',
  'category': ' 東京, 自転車・販売・買取・修理, 車・不動産, 閉店情報, 関東・甲信越',
  'category_tag': ['category-tokyo',
   'category-bicycle-shop',
   'category-car-estate',
   'category-close',
   'category-kantou_koushinetsu']},
 {'name': '【閉店】FLIPPER’S（フリッパーズ） 表参道店

## データの前処理
- pandasを使ってデータを前処理する
- pandas のかんたんなチュートリアルは こちらをどうぞ[10 minutes to pandas — pandas 1.3.2 documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html)
- いろいろ深く勉強したい人は [改訂版　Pythonユーザのための Jupyter［実践］入門](https://www.amazon.co.jp/dp/B08G1HYL9P) がオススメです




In [74]:
import pandas as pd


### なぜデータを辞書型にしてリストに格納したのか？

pandas の DataFrame クラスは、同じデータキーワードを持つデータのリストを渡すと、キーワードをコラムにしてDataFrameで返します。辞書型にしてリストのデータを作った理由はここにあります。


In [75]:
heitens = html_to_data(heiten_files)


In [77]:
df = pd.DataFrame(heitens)

In [79]:
# DataFrame オブジェクトの .head() メソッド：最初の五行だけ表示
df.head()

,name,post_time,url,category,category_tag
0,【閉店】串カツ田中 大森店,2021-02-16,https://kaiten-heiten.com/kushi-tanaka-omori/,"串揚げ・串焼き, 居酒屋, 東京, 閉店情報, 関東・甲信越, 飲食店","[category-kushiage, category-izakaya, category..."
1,【閉店】ドラッグストアスマイル下井草駅前店,2021-02-15,https://kaiten-heiten.com/drug-smile-igusaekimae/,"ドラッグストア・調剤薬局, 小売店, 東京, 閉店情報, 関東・甲信越","[category-drug-store, category-shop, category-..."
2,【閉店】RATIO&C,2021-02-15,https://kaiten-heiten.com/ratio-c/,"東京, 自転車・販売・買取・修理, 車・不動産, 閉店情報, 関東・甲信越","[category-tokyo, category-bicycle-shop, catego..."
3,【閉店】FLIPPER’S（フリッパーズ） 表参道店,2021-02-15,https://kaiten-heiten.com/flippers-omotesando/,"パンケーキ・クレープ, 東京, 閉店情報, 関東・甲信越, 飲食店","[category-pancake, category-tokyo, category-cl..."
4,【閉店】TOCCA STORE 銀座店,2021-02-15,https://kaiten-heiten.com/toccastore-ginza/,"小売店, 東京, 洋服, 閉店情報, 関東・甲信越","[category-shop, category-tokyo, category-cloth..."


In [81]:
# .info() メソッド。各コラムのデータ情報
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1344 entries, 0 to 1343
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          1344 non-null   object
 1   post_time     1344 non-null   object
 2   url           1344 non-null   object
 3   category      1344 non-null   object
 4   category_tag  1344 non-null   object
dtypes: object(5)
memory usage: 52.6+ KB


### 時系列データは、日時情報を index に持つのが吉

- 開店閉店情報は時系列情報です。よって日付け情報である `post_time` を時系列インデックスに変換していたほうがいろいろ幸せなことが多いと思います。
- `post_time` の型（⇑のDtype）は object になっています。DataFrameで object とは文字列のことです。
- これを Pandas の `to_datetime()` メソッドでDateTime型へ変換します。
- その後、DataFrame の `set_index()` メソッドで `post_time`をDataFrame のインデックスにします

In [84]:
# datetime64 型へ変換して上書き
df["post_time"] = pd.to_datetime(df["post_time"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1344 entries, 0 to 1343
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   name          1344 non-null   object        
 1   post_time     1344 non-null   datetime64[ns]
 2   url           1344 non-null   object        
 3   category      1344 non-null   object        
 4   category_tag  1344 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 52.6+ KB


In [86]:
# 見た目にはあまり変わらない
df.head()

,name,post_time,url,category,category_tag
0,【閉店】串カツ田中 大森店,2021-02-16,https://kaiten-heiten.com/kushi-tanaka-omori/,"串揚げ・串焼き, 居酒屋, 東京, 閉店情報, 関東・甲信越, 飲食店","[category-kushiage, category-izakaya, category..."
1,【閉店】ドラッグストアスマイル下井草駅前店,2021-02-15,https://kaiten-heiten.com/drug-smile-igusaekimae/,"ドラッグストア・調剤薬局, 小売店, 東京, 閉店情報, 関東・甲信越","[category-drug-store, category-shop, category-..."
2,【閉店】RATIO&C,2021-02-15,https://kaiten-heiten.com/ratio-c/,"東京, 自転車・販売・買取・修理, 車・不動産, 閉店情報, 関東・甲信越","[category-tokyo, category-bicycle-shop, catego..."
3,【閉店】FLIPPER’S（フリッパーズ） 表参道店,2021-02-15,https://kaiten-heiten.com/flippers-omotesando/,"パンケーキ・クレープ, 東京, 閉店情報, 関東・甲信越, 飲食店","[category-pancake, category-tokyo, category-cl..."
4,【閉店】TOCCA STORE 銀座店,2021-02-15,https://kaiten-heiten.com/toccastore-ginza/,"小売店, 東京, 洋服, 閉店情報, 関東・甲信越","[category-shop, category-tokyo, category-cloth..."


In [ ]:
# post_time を このDataFrameのインデックスにします
# .set_index() 
df.set_index("post_time", inplace=True)
